In [4]:
from math import radians, cos, sin, asin, sqrt
#from googleplaces import GooglePlaces, types, lang
import pandas as pd
import json
import math
import re
from ast import literal_eval
from time import sleep
from tqdm import trange
import pandasql as ps
import numpy as np

In [2]:
#Bearing Calculation
def bearing(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    # radius = 6371 # km

    rlat1 = math.radians(lat1)
    rlat2 = math.radians(lat2)
    rlon1 = math.radians(lon1)
    rlon2 = math.radians(lon2)
    dlon = math.radians(lon2-lon1)

    b = math.atan2(math.sin(dlon)*math.cos(rlat2),math.cos(rlat1)*math.sin(rlat2)-math.sin(rlat1)*math.cos(rlat2)*math.cos(dlon)) # bearing calc
    bd = math.degrees(b)
    br,bn = divmod(bd+360,360) # the bearing remainder and final bearing
    
    return bn

# Lat, Long of a destination based on bearing and given distance
def getDestinationLatLong(lat_long,azimuth,distance):
    lat,lng=lat_long
    '''returns the lat an long of destination point 
    given the start lat, long, aziuth, and distance'''
    R = 6378.1 #Radius of the Earth in km
    brng = math.radians(azimuth) #Bearing is degrees converted to radians.
    d = distance/1000 #Distance m converted to km
    lat1 = math.radians(lat) #Current dd lat point converted to radians
    lon1 = math.radians(lng) #Current dd long point converted to radians
    lat2 = math.asin(math.sin(lat1) * math.cos(d/R) + math.cos(lat1)* math.sin(d/R)* math.cos(brng))
    lon2 = lon1 + math.atan2(math.sin(brng) * math.sin(d/R)* math.cos(lat1), math.cos(d/R)- math.sin(lat1)* math.sin(lat2))
    #convert back to degrees
    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)
    return[lat2, lon2]

def getPathLength(lat_lng1,lat_lng2):
    '''calculates the distance between two lat, long coordinate pairs'''
    lat1,lng1=lat_lng1
    lat2,lng2=lat_lng2
    R = 6371000 # radius of earth in m
    lat1rads = math.radians(lat1)
    lat2rads = math.radians(lat2)
    deltaLat = math.radians((lat2-lat1))
    deltaLng = math.radians((lng2-lng1))
    a = math.sin(deltaLat/2) * math.sin(deltaLat/2) + math.cos(lat1rads) * math.cos(lat2rads) * math.sin(deltaLng/2) * math.sin(deltaLng/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d


def angleFromCoordinate(lat1, long1, lat2, long2):
    dLon = (long2 - long1)

    y = math.sin(dLon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dLon)

    brng = math.atan2(y, x)

    brng = math.degrees(brng)
    brng = (brng + 360) % 360
    brng = 360 - brng # count degrees clockwise - remove to make counter-clockwise

    return brng



def latlong_to_3d(latr, lonr):
    """Convert a point given latitude and longitude in radians to
    3-dimensional space, assuming a sphere radius of one."""
    return np.array((
        math.cos(latr) * math.cos(lonr),
        math.cos(latr) * math.sin(lonr),
        math.sin(latr)
    ))

def angle_between_vectors_degrees(u, v):
    """Return the angle between two vectors in any dimension space,
    in degrees."""
    return np.degrees(
        math.acos(np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))))

In [5]:
filtered_data=pd.read_excel('Restaurant.xlsx')

In [8]:
filtered_data.head(4)

,Rating,Latitude,Longitude,Name,Website,Vicinity,Address,Food Tags,Place tags
0,8.4,28.687150,76.965056,Red Chilli Chinese Food,0,"PVC Markat Road, Tikri Kalan, Delhi","PVC Markat Rd, Tikri Kalan, Delhi, 110041, India",0,0
1,7.4,28.678565,76.974541,Abdul Kadir,1,"Rohtak Road, Tikri Kalan, Delhi","Rohtak Rd, Tikri Kalan, Delhi, 110072, India",0,0
2,8.2,28.688256,76.950535,Veer Ji Restaurant & Banquet,0,"MIE Part-B, Bahadurgarh","MIE Part-B, Bahadurgarh, Haryana 124507, India",0,0
3,8.0,28.701612,76.976442,Babas' Snack bar,0,"Unnamed Road,, Delhi","Unnamed Road,, Delhi, 110081, India",1,0


In [6]:
x_coord=[]
y_coord=[]

In [7]:
A=(filtered_data.loc[0,'Latitude'],filtered_data.loc[0,'Longitude'])

In [9]:
A

(28.6871503, 76.9650556)

In [11]:
B = (28.57554, 76.84052)
C = (28.26236, 77.27081)

# Convert the points to numpy latitude/longitude radians space

b = np.radians(np.array(B))
c = np.radians(np.array(C))



# Adjust vectors for changed longitude scale at given latitude into 2D space

# Find the angle between the vectors in 2D space


In [14]:
for i in range(len(filtered_data)):
    A=(filtered_data.loc[i,'Latitude'],filtered_data.loc[i,'Longitude'])
    a = np.radians(np.array(A))
    # Vectors in latitude/longitude space
    avec = a - b
    cvec = c - b
    lat = b[0]
    avec[1] *= math.cos(lat)
    cvec[1] *= math.cos(lat)

    angle2deg = angle_between_vectors_degrees(avec, cvec)
    cos_th=math.cos(math.radians(angle2deg))
    sin_th=math.sin(math.radians(angle2deg))
    len_path=getPathLength(B,A)
    x_coord.append(len_path*cos_th)
    y_coord.append(len_path*sin_th)
    
    

In [ ]:
for i in trange(len(pre_clust), leave=True):
    A=(pre_clust.loc[i,'Latitude'],pre_clust.loc[i,'Longitude'])
    a = np.radians(np.array(A))
    # Vectors in latitude/longitude space
    avec = a - b
    cvec = c - b
    lat = b[0]
    avec[1] *= math.cos(lat)
    cvec[1] *= math.cos(lat)

    angle2deg = angle_between_vectors_degrees(avec, cvec)
    cos_th=math.cos(math.radians(angle2deg))
    sin_th=math.sin(math.radians(angle2deg))
    len_path=getPathLength(B,A)
    x_coord.append(len_path*cos_th)
    y_coord.append(len_path*sin_th)
    
    sleep(.00001)

In [16]:
# Converting meter to kilometer
filtered_data['X_coord']=np.array(x_coord)/1000
filtered_data['Y_coord']=np.array(y_coord)/1000

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
scalerx=MinMaxScaler()
scalery=MinMaxScaler()

In [19]:
clust=filtered_data.copy()

In [21]:
clust['X']=pd.DataFrame(scalerx.fit_transform(filtered_data['X_coord'].to_frame()),columns=['X'])

clust['Y']=pd.DataFrame(scalery.fit_transform(filtered_data['Y_coord'].to_frame()),columns=['Y'])


In [22]:
clust.head()

,Rating,Latitude,Longitude,Name,Website,Vicinity,Address,Food Tags,Place tags,X_coord,Y_coord,X,Y
0,8.4,28.687150,76.965056,Red Chilli Chinese Food,0,"PVC Markat Road, Tikri Kalan, Delhi","PVC Markat Rd, Tikri Kalan, Delhi, 110041, India",0,0,1.443391,17.310905,0.026825,0.374454
1,7.4,28.678565,76.974541,Abdul Kadir,1,"Rohtak Road, Tikri Kalan, Delhi","Rohtak Rd, Tikri Kalan, Delhi, 110072, India",0,0,2.765270,17.166695,0.049635,0.371302
2,8.2,28.688256,76.950535,Veer Ji Restaurant & Banquet,0,"MIE Part-B, Bahadurgarh","MIE Part-B, Bahadurgarh, Haryana 124507, India",0,0,0.273505,16.501552,0.006638,0.356761
3,8.0,28.701612,76.976442,Babas' Snack bar,0,"Unnamed Road,, Delhi","Unnamed Road,, Delhi, 110081, India",1,0,1.273346,19.257526,0.023891,0.417010
4,7.2,28.738093,77.007077,Bikaner Sweets,1,"kanjhawla chowk, Delhi","kanjhawla chowk, Chandpur Village, Delhi, 1100...",0,0,0.988135,24.286845,0.018970,0.526956


In [23]:
scaler_rating=MinMaxScaler()
clust['Rating_scaled']=pd.DataFrame(scalerx.fit_transform(filtered_data['Rating'].to_frame()),columns=['Rating_scaled'])

In [24]:
clust.head()

,Rating,Latitude,Longitude,Name,Website,Vicinity,Address,Food Tags,Place tags,X_coord,Y_coord,X,Y,Rating_scaled
0,8.4,28.687150,76.965056,Red Chilli Chinese Food,0,"PVC Markat Road, Tikri Kalan, Delhi","PVC Markat Rd, Tikri Kalan, Delhi, 110041, India",0,0,1.443391,17.310905,0.026825,0.374454,0.578947
1,7.4,28.678565,76.974541,Abdul Kadir,1,"Rohtak Road, Tikri Kalan, Delhi","Rohtak Rd, Tikri Kalan, Delhi, 110072, India",0,0,2.765270,17.166695,0.049635,0.371302,0.315789
2,8.2,28.688256,76.950535,Veer Ji Restaurant & Banquet,0,"MIE Part-B, Bahadurgarh","MIE Part-B, Bahadurgarh, Haryana 124507, India",0,0,0.273505,16.501552,0.006638,0.356761,0.526316
3,8.0,28.701612,76.976442,Babas' Snack bar,0,"Unnamed Road,, Delhi","Unnamed Road,, Delhi, 110081, India",1,0,1.273346,19.257526,0.023891,0.417010,0.473684
4,7.2,28.738093,77.007077,Bikaner Sweets,1,"kanjhawla chowk, Delhi","kanjhawla chowk, Chandpur Village, Delhi, 1100...",0,0,0.988135,24.286845,0.018970,0.526956,0.263158


In [25]:
cols_for_clustering=['Rating_scaled','Website','Food Tags','Place tags','X','Y']

In [26]:
clust_df=clust[cols_for_clustering]